In [1]:
import bilby
import numpy as np 
import time

C = 299792458.
G = 6.67408*1e-11
Mo = 1.989*1e30
Mpc = 3.086*1e22

In [2]:
%load_ext cython

In [2]:
import hphc

In [11]:
hphc

<module 'hphc' from '/home/hemantakumar.phurailatpam/bilby/eccentricity/cython/hphc.cpython-37m-x86_64-linux-gnu.so'>

In [3]:
def eccentric_waveform(frequency_array_, chirp_mass, mass_ratio, initial_eccentricity, luminosity_distance, theta_jn, psi, phase, geocent_time, ra, dec, **kwargs):
    
    mass_1 = (chirp_mass*(1+mass_ratio)**(1/5))/mass_ratio**(3/5)
    mass_2 = chirp_mass*mass_ratio**(2/5)*(1+mass_ratio)**(1/5)
    luminosity_distance = luminosity_distance*Mpc
    total_mass = (mass_1+mass_2)*Mo
    symmetric_mass_ratio = (mass_1*mass_2)/((mass_1+mass_2)**2)
    #last stable orbit
    lso_f = (C**3)/( G*(mass_1+mass_2)*Mo*np.pi*6**(3/2) )
    mass_diff = (mass_1-mass_2)*Mo
    f_min = 20.
    #f_max is set according to the prior we choose, depends on total_mass. 
    f_max = 100.
    #time of coalescence is taken to be 0 for convenience
    tc = 0.
    
    foo = np.array(frequency_array_, dtype='float')

    h_plus,h_cross = hphc.htilde(foo, total_mass, symmetric_mass_ratio, mass_diff, initial_eccentricity, luminosity_distance, theta_jn, psi, phase, tc, f_min, lso_f, f_max)

    return {'plus': h_plus, 'cross': h_cross}

In [2]:
import eccn.functions as fn

In [4]:
eccentric_waveform([50.],19.564163812778446,0.9148936170212766,0.1,200.0,0.4,0.1,1.2,1180002601.0,45,5.73)

{'plus': array([-1.95389735e-24+6.88698336e-22j]),
 'cross': array([-6.84333349e-22-2.26212308e-24j])}

In [5]:
t1 = time.time()

zz = fn.eccentric_waveform([50.],19.564163812778446,0.9148936170212766,0.1,200.0,0.4,0.1,1.2,1180002601.0,45,5.73)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.00384783744812011719


In [6]:
zz

{'plus': array([3.79312088e-23+1.41930529e-23j]),
 'cross': array([-1.41466021e-23+3.7803568e-23j])}

### cython speed up 2.7x
##### worth it !?

In [14]:
t1 = time.time()

zz = bilby.gw.source.lal_eccentric_binary_black_hole_no_spins( np.arange(20.,200.,1.) ,25.0,20.0,0.1,200.0,0.4,0.1)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.00252079963684082031


In [32]:
t1 = time.time()

zz = bilby.gw.source.lal_binary_black_hole( np.arange(20.,200.,0.1) ,25.0,20.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.00364470481872558594


In [17]:
bilby.gw.source.lal_binary_black_hole

<function bilby.gw.source.lal_binary_black_hole(frequency_array, mass_1, mass_2, luminosity_distance, a_1, tilt_1, phi_12, a_2, tilt_2, phi_jl, theta_jn, phase, **kwargs)>